In [1]:
import os

In [2]:
# Change the current working directory to the project root folder
# This ensures the code works with relative paths and avoids FileNotFoundError
os.chdir("../")
%pwd

'd:\\Projects\\Apple-Disease-Classification-Project'

In [3]:
import tensorflow as tf

In [4]:
from pathlib import Path
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    root_dir: Path
    trained_model_json_path: Path
    trained_model_weights_path: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [6]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        training = self.config.training
        model_preparation = self.config.model_preparation
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "dataset")
        create_directories([ Path(training.root_dir) ])


        eval_config = EvaluationConfig(
            root_dir=Path(training.root_dir),            
            trained_model_json_path=Path(training.trained_model_path).with_suffix(".json"), # = "artifacts/training/model.json"
            trained_model_weights_path=Path(training.trained_model_path).with_suffix(".h5"),
            training_data=Path(training_data),
            all_params=self.params,         
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




In [7]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop 
from pathlib import Path
from urllib.parse import urlparse

In [8]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def test_generator(self):
        test_datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=10,
            horizontal_flip=True
            )

        self.test_generator = test_datagen.flow_from_directory(
            f'{self.config.training_data}/test',
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            class_mode="sparse",
            shuffle=False,
            )
        
    def load_model(self):
        try:
            # Load model architecture
            with open(self.config.trained_model_json_path, 'r') as json_file:
                model_json = json_file.read()
            self.model = tf.keras.models.model_from_json(model_json)
            
            # Load model weights
            self.model.load_weights(self.config.trained_model_weights_path)

            ''' 
            When you load the model using tf.keras.models.load_model(path), the training configuration is loaded, 
            but sometimes it doesn't include the optimizer settings. Thus, you need to recompile it.

            Evaluation: Before evaluating the loaded model in model_evaluation.py, 
            recompiling ensures that the model has the necessary settings for loss and metrics to perform evaluation.
            '''

            # Compile the model
            optimizer = RMSprop(
                learning_rate=self.config.all_params.LEARNING_RATE,
                rho=self.config.all_params.RHO,
                epsilon=self.config.all_params.EPSILON
            )
            self.model.compile(
                optimizer=optimizer, 
                loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
                metrics=["accuracy"]
            )

        except Exception as e:
            raise RuntimeError("Error loading model architecture or weights") from e


    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)


    def evaluation(self):
        self.score = self.model.evaluate(self.test_generator)
        self.save_score()

In [ ]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.test_generator()
    evaluation.load_model()    
    evaluation.evaluation()
except Exception as e:
   raise e